# Notebook 01 — Data Exploration

## Formula 1 Race Performance Analysis (2018-2024)

---

| Step | Description |
|------|-------------|
| 1 | Load raw CSV files from Ergast F1 Database |
| 2 | Examine structure, columns, and data types |
| 3 | Analyze missing values and data quality |
| 4 | Understand relationships between tables |
| 5 | Create base merged dataset for preprocessing |

### Output

dataset saved to: `../data/processed/f1_base_2018_2024.csv`

---

### Table of Contents

1. [Setup & Imports](#1-setup)
2. [Load Raw Data](#2-load)
3. [Data Overview](#3-overview)
4. [Explore Each Table](#4-explore)
5. [Missing Values Analysis](#5-missing)
6. [Data Visualization](#6-visualization)
7. [Create Base Dataset](#7-merge)
8. [Save & Summary](#8-save)

---

## 1. Setup & Imports <a id='1-setup'></a>

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt


pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 50)


DATA_RAW = Path('../data/raw')
DATA_PROCESSED = Path('../data/processed')
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("Setup complete!")
print(f"Raw data folder: {DATA_RAW.resolve()}")
print(f"Processed folder: {DATA_PROCESSED.resolve()}")

In [ ]:
#checking available raw data files
print("Raw folder exists:", DATA_RAW.exists())
print()
print("Available CSV files:")
csv_files = sorted([p.name for p in DATA_RAW.glob('*.csv')])
for f in csv_files:
    print(f"   - {f}")
print(f"\nTotal: {len(csv_files)} files")

---

## 2. Load Raw Data <a id='2-load'></a>

 load the 5 main tables needed for our analysis:

| Table | Description | Key Columns |
|-------|-------------|-------------|
| `races.csv` | Race events info | raceId, year, name, date, circuitId |
| `results.csv` | Race results per driver | resultId, raceId, driverId, grid, position, points |
| `drivers.csv` | Driver information | driverId, forename, surname, nationality |
| `constructors.csv` | Team information | constructorId, name, nationality |
| `status.csv` | Finish status codes | statusId, status (Finished, DNF reason, etc.) |

In [ ]:
def load_csv(name: str) -> pd.DataFrame:
    """Load a CSV file from the raw data folder with error handling."""
    path = DATA_RAW / name
    if not path.exists():
        raise FileNotFoundError(f"Missing {path}. Please check your data/raw folder.")
    return pd.read_csv(path)

#main tables
races = load_csv('races.csv')
results = load_csv('results.csv')
drivers = load_csv('drivers.csv')
constructors = load_csv('constructors.csv')

#status table
status_path = DATA_RAW / 'status.csv'
status = pd.read_csv(status_path) if status_path.exists() else None

print("All tables loaded successfully!")

---

## 3. Data Overview <a id='3-overview'></a>

Let's get a high-level view of all tables before diving into details.

In [ ]:
#summary of all tables
tables = {
    'races': races,
    'results': results,
    'drivers': drivers,
    'constructors': constructors,
    'status': status
}

summary_data = []
for name, df in tables.items():
    if df is not None:
        summary_data.append({
            'Table': name,
            'Rows': df.shape[0],
            'Columns': df.shape[1],
            'Memory (KB)': round(df.memory_usage(deep=True).sum() / 1024, 1),
            'Missing Values': df.isna().sum().sum(),
            'Missing %': round(df.isna().sum().sum() / (df.shape[0] * df.shape[1]) * 100, 2)
        })

summary_df = pd.DataFrame(summary_data)
print("=" * 70)
print("DATA OVERVIEW - ALL TABLES")
print("=" * 70)
print()
display(summary_df)

In [ ]:
#show columns for each table
print("COLUMNS IN EACH TABLE:")
print("=" * 70)
for name, df in tables.items():
    if df is not None:
        print(f"\n{name.upper()} ({df.shape[1]} columns):")
        print(f"   {', '.join(df.columns.tolist())}")

---

## 4. Explore Each Table <a id='4-explore'></a>

examining each table in detail: first rows, data types, and basic statistics.

### 4.1 Races Table

Contains information about each F1 race event (Grand Prix).

In [ ]:
print("RACES TABLE")
print("=" * 50)
print(f"Shape: {races.shape[0]} rows x {races.shape[1]} columns")
print(f"Year range: {races['year'].min()} - {races['year'].max()}")
print(f"Unique circuits: {races['circuitId'].nunique()}")
print()
races.head()

In [ ]:
#Data types
print("Data Types:")
print(races.dtypes)

### 4.2 Results Table

contains race results for each driver in each race. this is **main table**.

In [ ]:
print("RESULTS TABLE")
print("=" * 50)
print(f"Shape: {results.shape[0]} rows x {results.shape[1]} columns")
print(f"Unique races: {results['raceId'].nunique()}")
print(f"Unique drivers: {results['driverId'].nunique()}")
print(f"Points range: {results['points'].min()} - {results['points'].max()}")
print()
results.head()

In [ ]:
# Data types
print("Data Types:")
print(results.dtypes)

### 4.3 Drivers Table

Contains driver biographical information.

In [ ]:
print("DRIVERS TABLE")
print("=" * 50)
print(f"Shape: {drivers.shape[0]} rows x {drivers.shape[1]} columns")
print(f"Unique nationalities: {drivers['nationality'].nunique()}")
print()
drivers.head()

In [ ]:
# Data types
print("Data Types:")
print(drivers.dtypes)

### 4.4 Constructors Table

Contains team (constructor) information.

In [ ]:
print("CONSTRUCTORS TABLE")
print("=" * 50)
print(f"Shape: {constructors.shape[0]} rows x {constructors.shape[1]} columns")
print(f"Unique nationalities: {constructors['nationality'].nunique()}")
print()
constructors.head()

In [ ]:
# Data types
print("Data Types:")
print(constructors.dtypes)

### 4.5 Status Table

Contains finish status codes (Finished, DNF reasons, etc.).

In [ ]:
if status is not None:
    print("STATUS TABLE")
    print("=" * 50)
    print(f"Shape: {status.shape[0]} rows x {status.shape[1]} columns")
    print()
    print("Sample status codes:")
    display(status.head(15))
else:
    print("Status table not available.")

---

## 5. Missing Values Analysis <a id='5-missing'></a>

Understanding missing data is critical for preprocessing decisions.

In [ ]:
def missing_report(df: pd.DataFrame, table_name: str = "Table") -> pd.DataFrame:
    """Generate a missing values report for a dataframe."""
    missing = df.isna().sum()
    missing_pct = (df.isna().sum() / len(df) * 100).round(2)
    
    report = pd.DataFrame({
        'Column': missing.index,
        'Missing Count': missing.values,
        'Missing %': missing_pct.values,
        'Dtype': df.dtypes.values
    })
    
    return report[report['Missing Count'] > 0].sort_values('Missing %', ascending=False)

In [ ]:
#missing values in Results table(important)
print("MISSING VALUES - RESULTS TABLE")
print("=" * 50)
missing_results = missing_report(results, 'results')
if len(missing_results) > 0:
    display(missing_results)
else:
    print("No missing values in results table!")

In [ ]:
#missing values in Races table
print("MISSING VALUES - RACES TABLE")
print("=" * 50)
missing_races = missing_report(races, 'races')
if len(missing_races) > 0:
    display(missing_races.head(10))
else:
    print("No missing values in races table!")

In [ ]:
#check for special values like \N (Ergast uses this for NULL)
print("SPECIAL VALUES CHECK")
print("=" * 50)
print("\nChecking for '\\N' values (Ergast NULL marker):")

for name, df in [('results', results), ('races', races)]:
    backslash_n_count = (df == '\\N').sum().sum()
    if backslash_n_count > 0:
        print(f"   {name}: {backslash_n_count} '\\N' values found")
        cols_with_backslash = df.columns[(df == '\\N').any()].tolist()
        print(f"   Columns: {cols_with_backslash}")
    else:
        print(f"   {name}: No '\\N' values")

---

## 6. Data Visualization <a id='6-visualization'></a>

visualizations to understand the data distribution.

In [ ]:
#Races per year
races_per_year = races.groupby('year').size()

fig, ax = plt.subplots(figsize=(12, 5))
races_per_year.plot(kind='bar', ax=ax, color="#B92214", edgecolor='black')
ax.set_title('Number of Races per Year (All F1 History)', fontsize=14, fontweight='bold')
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Number of Races', fontsize=12)
ax.axhline(y=races_per_year.mean(), color='black', linestyle='--', label=f'Average: {races_per_year.mean():.1f}')
ax.legend()

#Show only every 5th year label
tick_positions = range(0, len(races_per_year), 5)
ax.set_xticks(tick_positions)
ax.set_xticklabels(races_per_year.index[::5], rotation=45)

plt.tight_layout()
plt.show()

print(f"\nRecent years (2018-2024): {races_per_year.loc[2018:2024].sum()} races")

In [ ]:
#Points distribution in results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

#points histogram
axes[0].hist(results['points'], bins=30, color="#c7301c", edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Race Points (All Results)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Points')
axes[0].set_ylabel('Frequency')

#grid position histogram
grid_clean = pd.to_numeric(results['grid'], errors='coerce')
axes[1].hist(grid_clean.dropna(), bins=20, color="#eb9e11", edgecolor='black', alpha=0.7)
axes[1].set_title('Distribution of Grid Positions', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Grid Position')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
#top 10 driver nationalities
top_nationalities = drivers['nationality'].value_counts().head(10)

fig, ax = plt.subplots(figsize=(10, 5))
top_nationalities.plot(kind='barh', ax=ax, color='#38383f', edgecolor='black')
ax.set_title('Top 10 Driver Nationalities (All Time)', fontsize=14, fontweight='bold')
ax.set_xlabel('Number of Drivers')
ax.set_ylabel('Nationality')
ax.invert_yaxis()

plt.tight_layout()
plt.show()

---

## 7. Create Base Dataset <a id='7-merge'></a>

merge the tables to create a base dataset for preprocessing.

### Merge Strategy

```
results (main table)
    |-- JOIN races ON raceId        -> Get year, date, race name
    |-- JOIN drivers ON driverId    -> Get driver name, nationality
    +-- JOIN constructors ON constructorId -> Get team name
```

### Filter: 2018-2024 Only

focus on the modern hybrid era for more relevant analysis.

In [ ]:
#Step 1:filter races to 2018-2024
races_filtered = races.loc[
    (races['year'] >= 2018) & (races['year'] <= 2024),
    ['raceId', 'year', 'round', 'name', 'date']
].copy()

print(f"Races filtered to 2018-2024: {len(races_filtered)} races")
print(f"Years: {races_filtered['year'].min()} - {races_filtered['year'].max()}")

In [ ]:
#step 2: merge results with filtered races
base = results.merge(races_filtered, on='raceId', how='inner')
print(f"After merging with races: {len(base)} rows")

# step 3:prepare driver names
drivers_small = drivers[['driverId', 'forename', 'surname', 'nationality']].copy()
drivers_small['driverName'] = drivers_small['forename'].astype(str) + ' ' + drivers_small['surname'].astype(str)

#step 4:merge with drivers
base = base.merge(
    drivers_small[['driverId', 'driverName', 'nationality']], 
    on='driverId', 
    how='left'
)
print(f"After merging with drivers: {len(base)} rows")

#step 5:Prepare constructor names
constructors_small = constructors[['constructorId', 'name']].copy()
constructors_small = constructors_small.rename(columns={'name': 'constructorName'})

#step 6:merge with constructors
base = base.merge(
    constructors_small[['constructorId', 'constructorName']], 
    on='constructorId', 
    how='left'
)
print(f"After merging with constructors: {len(base)} rows")

In [ ]:
#step 7:select and order columns for the base dataset
keep_cols = [
    'raceId', 'year', 'round', 'name', 'date',           #race info
    'driverId', 'driverName', 'nationality',              #driver info
    'constructorId', 'constructorName',                   # team info
    'grid', 'positionOrder', 'points',                    #results
]

#adding statusId if available
if 'statusId' in base.columns:
    keep_cols.append('statusId')

base = base[keep_cols].copy()

#convert date to datetime
base['date'] = pd.to_datetime(base['date'], errors='coerce')

print(f"\nBase dataset created!")
print(f"Shape: {base.shape[0]} rows x {base.shape[1]} columns")

In [ ]:
# Preview base dataset
print("BASE DATASET PREVIEW")
print("=" * 50)
base.head(10)

In [ ]:
# Data types and info
print("BASE DATASET INFO")
print("=" * 50)
base.info()

In [ ]:
# Statistics
print("BASE DATASET STATISTICS")
print("=" * 50)
display(base.describe(include='all').T)

In [ ]:
#summary of the base dataset
print("BASE DATASET SUMMARY")
print("=" * 50)
print(f"Total records:      {len(base):,}")
print(f"Unique races:       {base['raceId'].nunique()}")
print(f"Unique drivers:     {base['driverId'].nunique()}")
print(f"Unique teams:       {base['constructorId'].nunique()}")
print(f"Year range:         {base['year'].min()} - {base['year'].max()}")
print(f"Date range:         {base['date'].min().date()} to {base['date'].max().date()}")
print(f"Points range:       {base['points'].min()} - {base['points'].max()}")
print(f"Grid positions:     {int(base['grid'].min())} - {int(base['grid'].max())}")

---

## 8. Save & Summary <a id='8-save'></a>

In [ ]:
#saving base dataset
OUT_PATH = DATA_PROCESSED / 'f1_base_2018_2024.csv'
base.to_csv(OUT_PATH, index=False)

print(f"Base dataset saved to: {OUT_PATH.resolve()}")
print(f"File size: {OUT_PATH.stat().st_size / 1024:.1f} KB")

In [ ]:
#Final summary
print("=" * 70)
print("NOTEBOOK 01 COMPLETE - DATA EXPLORATION SUMMARY")
print("=" * 70)
print()
print("RAW DATA LOADED:")
print(f"   - races.csv:        {races.shape[0]:,} rows")
print(f"   - results.csv:      {results.shape[0]:,} rows")
print(f"   - drivers.csv:      {drivers.shape[0]:,} rows")
print(f"   - constructors.csv: {constructors.shape[0]:,} rows")
print(f"   - status.csv:       {status.shape[0] if status is not None else 'N/A':,} rows")
print()
print("BASE DATASET CREATED:")
print(f"   - Records:  {base.shape[0]:,} driver-race entries")
print(f"   - Columns:  {base.shape[1]}")
print(f"   - Years:    2018-2024 (modern hybrid era)")
print(f"   - Output:   {OUT_PATH.name}")
print()
print("NEXT STEP:")
print("   -> Notebook 02: Data Preprocessing & Feature Engineering")
print()
print("=" * 70)